In [1]:
import numpy as np
import random
import tensorflow as tf
import os
import time
#https://www.tensorflow.org/tutorials/text/text_generation

## The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

In [33]:
opponent_history=["S", "R", "P", "S", "S", "R", "R", "P", "S", "S", "R", "S", "R", "R", "P"]
text=opponent_history  # easy to type

In [34]:
# Vectorize the text - map chars to a numerical representation
char2idx = { "R":0, "P":1, "S":2 }
#idx2char = {0:"R", 1:"P", 2:"S"}
idx2char = np.array(opponent_history)
print(type(idx2char))

oh_as_int = [char2idx[cc] for cc in opponent_history]
print(oh_as_int)

<class 'numpy.ndarray'>
[2, 0, 1, 2, 2, 0, 0, 1, 2, 2, 0, 2, 0, 0, 1]


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.
<p>
<b>
For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.
</b>
So break the text into chunks of seq_length+1. For example, say seq_length is 2 and our text is "SRPSSR". The input sequence would be "SR", and the target sequence "RP".


In [35]:
# The maximum length sentence we want for a single input in characters
seq_length = 3
examples_per_epoch = len(text)//(seq_length+1)

print("examples_per_epoch: ", examples_per_epoch)


examples_per_epoch:  3


In [36]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(oh_as_int)

for i in char_dataset.take(5):
  print(i.numpy(), idx2char[i.numpy()])


2 P
0 S
1 R
2 P
2 P


In [46]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

print("sequences type:", sequences)

for item in sequences.take(3):
  print(repr(''.join(idx2char[item.numpy()])))
  print(item.numpy())

sequences type: <BatchDataset shapes: (4,), types: tf.int32>
'PSRP'
[2 0 1 2]
'PSSR'
[2 0 0 1]
'PPSP'
[2 2 0 2]


In [64]:
# For each sequence, duplicate and shift it to form the input and target text by using 
# the map method to apply a simple function to each batch:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)


In [55]:
# Print the first examples input and target values:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))



[2 0 1]
Input data:  'PSR'
Target data: 'SRP'


In [17]:
# This is what we want to predict - for every input, print expected output. This is what
# we will train the model  with.
for i, (input_idx, target_idx) in enumerate(zip(input_example[:3], target_example[:3])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))


Step    0
  input: 2 ('P')
  expected output: 0 ('S')
Step    1
  input: 0 ('S')
  expected output: 1 ('R')
Step    2
  input: 1 ('R')
  expected output: 2 ('P')


## Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [57]:
# Batch size
BATCH_SIZE = 4

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset


<BatchDataset shapes: ((4, 3), (4, 3)), types: (tf.int32, tf.int32)>

## Build the Model

In [58]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [59]:
# Number of Unique Inputs
vocab_size = 3

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


In [66]:
model = build_model(
    vocab_size = 3,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)


In [61]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (4, None, 256)            768       
_________________________________________________________________
gru_3 (GRU)                  (4, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (4, None, 3)              3075      
Total params: 3,942,147
Trainable params: 3,942,147
Non-trainable params: 0
_________________________________________________________________


## Try the Model

In [67]:
# First check the shape of the output:

for input_example_batch, target_example_batch in dataset.take(2):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


ValueError: Shape (256, 3) must have rank at least 3

In [22]:
{str(ii):float(vv) for ii,vv in enumerate(range(11, 20))}

{'0': 11.0,
 '1': 12.0,
 '2': 13.0,
 '3': 14.0,
 '4': 15.0,
 '5': 16.0,
 '6': 17.0,
 '7': 18.0,
 '8': 19.0}

In [3]:
[ [c,c+1] for c in range(1,10)]

[[1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10]]

In [8]:
[ c for c in range(1,10) if c % 2 == 0]

[2, 4, 6, 8]

In [23]:
[ lambda c: c+1 for c in range(1,10) if c % 3 == 0]

[<function __main__.<listcomp>.<lambda>(c)>,
 <function __main__.<listcomp>.<lambda>(c)>,
 <function __main__.<listcomp>.<lambda>(c)>]

In [13]:
[ c:c for c in range(1,10) if c % 3 == 0]

SyntaxError: invalid syntax (<ipython-input-13-25ee24e2ddb0>, line 1)

In [26]:
(5 % 2 and 'odd' or 'even')

'odd'

In [27]:
ii = 10

In [28]:
type(ii)

int

In [29]:
ii = 10.0

In [30]:
type(ii)

float

In [2]:
x = lambda arg: print(arg)
x("help")

help
